In [36]:
import pandas as pd
import numpy as np

In [3]:
raw_data_folder = "../data/ratings.dat"

In [5]:
ratings = pd.read_csv(raw_data_folder, names=["u","v","r"], nrows=1000)

In [188]:
train_data_path =  "../data/matches_train_for_d2v.dat"
test_data_path =  "../data/matches_test.dat"

In [192]:
def load_d2v_formated_data(data_path):
    df = pd.read_csv(data_path)
    df["rated"] = df["rated"].map(ast.literal_eval)
    return df
df = load_d2v_formated_data(train_data_path) # rater, rated, r, m

In [183]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=df["rated"].values, size=100, window=5, min_count=1, workers=4)

In [184]:
model.train_count

1

In [199]:
model = Word2Vec(size=100, window=5, min_count=1, workers=4)
if model.train_count == 0:
    model.build_vocab(df["rated"].values)
model.train(df["rated"].values, total_examples=model.corpus_count)

TypeError: unhashable type: 'list'

In [196]:
df["rated"].values[0]

['971',
 '1616',
 '4633',
 '8305',
 '9345',
 '9729',
 '10148',
 '13210',
 '14604',
 '14985',
 '18032',
 '18287',
 '18345',
 '19006',
 '19727',
 '21256',
 '22319',
 '22915',
 '24294',
 '24683',
 '26084',
 '33960',
 '34406',
 '35647',
 '37773',
 '38356',
 '39414',
 '40377',
 '43836',
 '47314',
 '49466',
 '53933',
 '54929',
 '61157',
 '63063',
 '64257',
 '65602',
 '69882',
 '73281',
 '73611',
 '75050',
 '75243',
 '83622',
 '87068',
 '88179',
 '88886',
 '91102',
 '93105',
 '97655',
 '98556',
 '105720',
 '113916',
 '116926',
 '120342',
 '121859',
 '127833',
 '130970',
 '131669',
 '138830',
 '140246',
 '144218']

In [143]:
model.wv["133"] - model.wv["971"]

array([-0.12723374, -0.02117734,  0.08631784, -0.04747325,  0.00435691,
        0.03933713,  0.11287095, -0.11574654, -0.20549986, -0.2265603 ,
       -0.23648295, -0.04263033, -0.2536514 ,  0.23384523, -0.03672279,
       -0.01631407, -0.10842631,  0.06156471, -0.14064027,  0.05527634,
        0.06275773,  0.06318504, -0.09959326,  0.00712821, -0.08789454,
       -0.00228224,  0.1091316 , -0.19156131,  0.00605597, -0.0457546 ,
        0.03622174,  0.13340448, -0.17070979, -0.14861462,  0.01686485,
       -0.20882046,  0.10756664, -0.29366484,  0.03064293, -0.05771504,
       -0.07539541, -0.01506608, -0.18282723,  0.02333215,  0.04041461,
        0.0650878 , -0.29210657, -0.20053175, -0.21113163,  0.04593519,
       -0.05706215,  0.08994237, -0.03851984,  0.04202349,  0.0114984 ,
       -0.11510144, -0.1103283 , -0.00689706, -0.02189345, -0.10100105,
       -0.22201777,  0.06885063,  0.18893233, -0.18214199,  0.11545525,
        0.10921384,  0.02011329, -0.07345657, -0.04266724, -0.27

In [168]:
def load_d2v_formated_data(data_path):
    df = pd.read_csv(data_path)
    df["rated"] = df["rated"].map(ast.literal_eval)
    return df
df = load_d2v_formated_data(train_data_path)

In [203]:
def list_shuffler(x):
    np.random.shuffle(x)
    return x
df["rated"].apply(list_shuffler)

0      [971, 35647, 87068, 13210, 9345, 127833, 14024...
1      [5577, 12243, 25083, 55238, 7006, 134835, 6206...
2                         [25083, 130815, 93891, 105720]
3      [27357, 7964, 6499, 136297, 11878, 174211, 132...
4      [102366, 3015, 12107, 91366, 135266, 87789, 25...
                             ...                        
995    [78597, 85704, 33216, 67265, 8196, 13029, 5505...
996    [25083, 147908, 91366, 59874, 115192, 12795, 7...
997          [102751, 3446, 12845, 53583, 71636, 112075]
998    [115129, 135266, 12227, 24734, 163591, 26193, ...
999    [68140, 105522, 115872, 2111, 70720, 45418, 31...
Name: rated, Length: 1000, dtype: object

In [204]:
df["rated"]

0      [971, 35647, 87068, 13210, 9345, 127833, 14024...
1      [5577, 12243, 25083, 55238, 7006, 134835, 6206...
2                         [25083, 130815, 93891, 105720]
3      [27357, 7964, 6499, 136297, 11878, 174211, 132...
4      [102366, 3015, 12107, 91366, 135266, 87789, 25...
                             ...                        
995    [78597, 85704, 33216, 67265, 8196, 13029, 5505...
996    [25083, 147908, 91366, 59874, 115192, 12795, 7...
997          [102751, 3446, 12845, 53583, 71636, 112075]
998    [115129, 135266, 12227, 24734, 163591, 26193, ...
999    [68140, 105522, 115872, 2111, 70720, 45418, 31...
Name: rated, Length: 1000, dtype: object